In [257]:
import pandas as pd
from datetime import datetime

In [258]:
house = pd.read_csv("./loren_data/house_2022-07-15.csv")
house.head()

,disclosure_year,disclosure_date,transaction_date,owner,ticker,asset_description,type,amount,representative,district,ptr_link,cap_gains_over_200_usd
0,2021,10/04/2021,2021-09-27,joint,BP,BP plc,purchase,"$1,001 - $15,000",Hon. Virginia Foxx,NC05,https://disclosures-clerk.house.gov/public_dis...,False
1,2021,10/04/2021,2021-09-13,joint,XOM,Exxon Mobil Corporation,purchase,"$1,001 - $15,000",Hon. Virginia Foxx,NC05,https://disclosures-clerk.house.gov/public_dis...,False
2,2021,10/04/2021,2021-09-10,joint,ILPT,Industrial Logistics Properties Trust - Common...,purchase,"$15,001 - $50,000",Hon. Virginia Foxx,NC05,https://disclosures-clerk.house.gov/public_dis...,False
3,2021,10/04/2021,2021-09-28,joint,PM,Phillip Morris International Inc,purchase,"$15,001 - $50,000",Hon. Virginia Foxx,NC05,https://disclosures-clerk.house.gov/public_dis...,False
4,2021,10/04/2021,2021-09-17,self,BLK,BlackRock Inc,sale_partial,"$1,001 - $15,000",Hon. Alan S. Lowenthal,CA47,https://disclosures-clerk.house.gov/public_dis...,False


In [259]:
house.dtypes

disclosure_year            int64
disclosure_date           object
transaction_date          object
owner                     object
ticker                    object
asset_description         object
type                      object
amount                    object
representative            object
district                  object
ptr_link                  object
cap_gains_over_200_usd      bool
dtype: object

In [260]:
house.isnull().sum()

disclosure_year              0
disclosure_date              0
transaction_date             0
owner                     5614
ticker                       0
asset_description            4
type                         0
amount                       0
representative               0
district                     0
ptr_link                     0
cap_gains_over_200_usd       0
dtype: int64

In [261]:
house[house['owner'].isnull()].head()

,disclosure_year,disclosure_date,transaction_date,owner,ticker,asset_description,type,amount,representative,district,ptr_link,cap_gains_over_200_usd
6,2021,12/01/2021,2021-11-30,NaN,KPLTW,Katapult Holdings Inc - Warrant,purchase,"$1,001 - $15,000",Hon. Austin Scott,GA08,https://disclosures-clerk.house.gov/public_dis...,False
7,2021,12/01/2021,2021-11-18,NaN,AMD,Advanced Micro Devices Inc,sale_full,"$50,001 - $100,000",Hon. Thomas Suozzi,NY03,https://disclosures-clerk.house.gov/public_dis...,False
8,2021,12/01/2021,2021-11-18,NaN,AAPL,Apple Inc,sale_full,"$50,001 - $100,000",Hon. Thomas Suozzi,NY03,https://disclosures-clerk.house.gov/public_dis...,False
9,2021,12/01/2021,2021-11-24,NaN,MSFT,Microsoft Corporation,purchase,"$50,001 - $100,000",Hon. Thomas Suozzi,NY03,https://disclosures-clerk.house.gov/public_dis...,False
10,2021,12/01/2021,2021-11-24,NaN,MS,Morgan Stanley,purchase,"$100,001 - $250,000",Hon. Thomas Suozzi,NY03,https://disclosures-clerk.house.gov/public_dis...,False


In [262]:
percent_null = house.owner.isnull().sum()/len(house)
percent_null

0.3779198922921575

In [263]:
house['owner'].value_counts(dropna = False)

NaN          5614
joint        4635
self         2897
--           1315
dependent     394
Name: owner, dtype: int64

We decided to drop the `'owner'` column because there were so many nulls and '--'. For our purposes, to see if congresspeople are taking advantage of insider trading, the distinction of who in the congressperson's family technically owns the equity is unimportant. It would be interesting to explore ownership, but there's just too many nulls.

In [264]:
house.drop(columns = ['owner'], inplace = True)

In [265]:
house['disclosure_date'] = pd.to_datetime(house['disclosure_date'], yearfirst=True)

We discovered some wonky years in the transaction dates. Fortunately, it was highly likely the disclosure year was the year of the transaction, so we substituted that for the year

This [stackoverflow answer](https://stackoverflow.com/a/56968849) helped to understand how to pull the first item from within the list.

In [266]:
house['transaction_date'] = house['transaction_date'].str.split('-')

weird_years = house[(house['transaction_date'].str[0] != '2017') &
      (house['transaction_date'].str[0] != '2018') &
      (house['transaction_date'].str[0] != '2019') &
      (house['transaction_date'].str[0] != '2020') &
      (house['transaction_date'].str[0] != '2021') &
      (house['transaction_date'].str[0] != '2022')]

for i in weird_years.index:
    house.loc[i, 'transaction_date'][0] = str(house.loc[i, 'disclosure_year'])

In [267]:
#verifying the years have been fixed
house[(house['transaction_date'].str[0] != '2017') &
      (house['transaction_date'].str[0] != '2018') &
      (house['transaction_date'].str[0] != '2019') &
      (house['transaction_date'].str[0] != '2020') &
      (house['transaction_date'].str[0] != '2021') &
      (house['transaction_date'].str[0] != '2022')]

,disclosure_year,disclosure_date,transaction_date,ticker,asset_description,type,amount,representative,district,ptr_link,cap_gains_over_200_usd


In [268]:
house['transaction_date'] = house['transaction_date'].str.join('-')
house['transaction_date'] = pd.to_datetime(house['transaction_date'])
house.dtypes

disclosure_year                    int64
disclosure_date           datetime64[ns]
transaction_date          datetime64[ns]
ticker                            object
asset_description                 object
type                              object
amount                            object
representative                    object
district                          object
ptr_link                          object
cap_gains_over_200_usd              bool
dtype: object

In [269]:
#verifying the range of dates is what we expect
house['transaction_date'].describe(datetime_is_numeric=True)

count                            14855
mean     2020-11-29 17:24:47.256815616
min                2017-09-05 00:00:00
25%                2020-04-22 00:00:00
50%                2020-11-19 00:00:00
75%                2021-06-08 00:00:00
max                2022-12-31 00:00:00
Name: transaction_date, dtype: object

In [270]:
house['amount'].value_counts()

$1,001 - $15,000            10447
$15,001 - $50,000            2362
$50,001 - $100,000            750
$100,001 - $250,000           572
$250,001 - $500,000           243
$1,001 -                      242
$500,001 - $1,000,000         150
$1,000,001 - $5,000,000        41
$1,000,000 +                   30
$5,000,001 - $25,000,000        9
$1,000 - $15,000                4
$15,000 - $50,000               3
$50,000,000 +                   1
$1,000,000 - $5,000,000         1
Name: amount, dtype: int64

Review of .replace() from [here](https://www.symbiosisacademy.org/tutorial-index/pandas-search-replace-values-columns/)

In [271]:
house = house.replace(to_replace={'amount':['$1,001 -', '$1,000 - $15,000']}, value= '$1,001 - $15,000')
house = house.replace(to_replace={'amount':['$1,000,000 +', '$1,000,000 - $5,000,000']}, value= '$1,000,001 - $5,000,000')
house = house.replace(to_replace={'amount':['$15,000 - $50,000']}, value= '$15,001 - $50,000')
house['amount'].value_counts()

$1,001 - $15,000            10693
$15,001 - $50,000            2365
$50,001 - $100,000            750
$100,001 - $250,000           572
$250,001 - $500,000           243
$500,001 - $1,000,000         150
$1,000,001 - $5,000,000        72
$5,000,001 - $25,000,000        9
$50,000,000 +                   1
Name: amount, dtype: int64

In [272]:
house['amount'] = house['amount'].str.split(' ') 

for i in range(0, len(house)):
    if len(house.loc[i, 'amount']) == 3:
        house.loc[i, 'amount'] = house.loc[i, 'amount'][2]
    else:
        house.loc[i, 'amount'] = house.loc[i, 'amount'][0]
        
house['amount'] = house['amount'].map(lambda x: x.replace('$', '').replace(',', ''))
house['amount'] = house['amount'].astype(int)
house['amount'].value_counts()

15000       10693
50000        2365
100000        750
250000        572
500000        243
1000000       150
5000000        72
25000000        9
50000000        1
Name: amount, dtype: int64

In [273]:
house.dtypes

disclosure_year                    int64
disclosure_date           datetime64[ns]
transaction_date          datetime64[ns]
ticker                            object
asset_description                 object
type                              object
amount                             int64
representative                    object
district                          object
ptr_link                          object
cap_gains_over_200_usd              bool
dtype: object

In [274]:
house['ticker'].value_counts()

--       1208
MSFT      244
AAPL      181
NTAP      130
TDDXX     122
         ... 
MSTR        1
MC          1
MPWR        1
HEI         1
FLCB        1
Name: ticker, Length: 2123, dtype: int64

In [275]:
house['ticker'].value_counts().values[0]/len(house)

0.08131942107034669

In [276]:
house[house['ticker'] == '--'].head()

,disclosure_year,disclosure_date,transaction_date,ticker,asset_description,type,amount,representative,district,ptr_link,cap_gains_over_200_usd
56,2020,2020-09-22,2020-08-17,--,Metallic Minerals Corp.,sale_partial,250000,Mr. TJ John (Tj) Cox,CA21,https://disclosures-clerk.house.gov/public_dis...,True
57,2021,2021-03-23,2021-01-27,--,Zimmer Biomet Holdings,sale_partial,15000,Hon. Alan S. Lowenthal,CA47,https://disclosures-clerk.house.gov/public_dis...,False
58,2021,2021-03-23,2021-02-08,--,Zimmer Biomet Holdings,sale_full,15000,Hon. Alan S. Lowenthal,CA47,https://disclosures-clerk.house.gov/public_dis...,False
59,2021,2021-03-23,2021-02-08,--,Zimmer Biomet Holdings,purchase,15000,Hon. Alan S. Lowenthal,CA47,https://disclosures-clerk.house.gov/public_dis...,False
61,2021,2021-03-23,2021-02-19,--,Celegene Corp,sale_full,50000,Hon. Alan S. Lowenthal,CA47,https://disclosures-clerk.house.gov/public_dis...,False


In [277]:
tickers = pd.read_excel('./loren_data/US-Stock-Symbols.xlsx')
tickers = tickers[['Symbol', 'Name']]
tickers.columns = tickers.columns.map(lambda x: x.lower())

tickers.head()

,symbol,name
0,AAPL,Apple Inc.
1,XOM,Exxon Mobil Corporation
2,MSFT,Microsoft Corporation
3,BAC^I,Bank of America Corporation
4,IBM,International Business Machines Corporation


In [278]:
for i in house.loc[house['ticker'] == '--']['asset_description']:
    symbol = tickers.loc[tickers['name'] == i, 'symbol']
    house.loc[house['asset_description'] == i, 'ticker'] = symbol

In [279]:
house.ticker.value_counts()

MSFT     244
AAPL     181
NTAP     130
TDDXX    122
FB       115
        ... 
MGPI       1
MSTR       1
MC         1
MPWR       1
FLCB       1
Name: ticker, Length: 2117, dtype: int64

In [280]:
house[house['ticker'] == '--']

,disclosure_year,disclosure_date,transaction_date,ticker,asset_description,type,amount,representative,district,ptr_link,cap_gains_over_200_usd


In [281]:
house.describe()

,disclosure_year,amount
count,14855.000000,1.485500e+04
mean,2020.634938,9.445608e+04
std,0.703684,8.227438e+05
min,2020.000000,1.500000e+04
25%,2020.000000,1.500000e+04
50%,2021.000000,1.500000e+04
75%,2021.000000,5.000000e+04
max,2022.000000,5.000000e+07


We need our columns to agree with the Senate dataframe columns, so we made the following changes. While we erred on the side of keeping information for now, we dropped the 'ptr_link' because we don't anticipate doing any analysis on it. If we need it, we'll come back and reference it from the csv or from prior to this point.

This [stackoverflow answer](https://stackoverflow.com/a/54195568) helped us to find the differences between the lists of columns 

In [282]:
house_cols = list(house.columns)

senate_cols = ['transaction_date', 'owner', 'ticker', 'asset_description',
       'asset_type', 'type', 'amount', 'comment', 'senator', 'ptr_link',
       'disclosure_date', 'year'] 

house_different_cols = list(set(house_cols).difference(senate_cols))
print(f'In House, not in Senate: {house_different_cols}')

senate_different_cols = list(set(senate_cols).difference(house_cols))
print(f'In Senate, not in House: {senate_different_cols}')

In House, not in Senate: ['representative', 'district', 'disclosure_year', 'cap_gains_over_200_usd']
In Senate, not in House: ['senator', 'owner', 'asset_type', 'year', 'comment']


In [283]:
house['chamber'] = 'house'
house.drop(columns = ['ptr_link', 'cap_gains_over_200_usd', 'disclosure_year'], inplace = True)
house.rename(columns = {'representative': 'name', 'district': 'represents'}, inplace = True)

In [284]:
house_cols = list(house.columns)

senate_cols = ['transaction_date', 'owner', 'ticker', 'asset_description',
       'asset_type', 'type', 'amount', 'comment', 'senator', 'ptr_link',
       'disclosure_date', 'year'] 

house_different_cols = list(set(house_cols).difference(senate_cols))
print(f'In House, not in Senate: {house_different_cols}')

senate_different_cols = list(set(senate_cols).difference(house_cols))
print(f'In Senate, not in House: {senate_different_cols}')

In House, not in Senate: ['name', 'chamber', 'represents']
In Senate, not in House: ['senator', 'owner', 'ptr_link', 'asset_type', 'year', 'comment']


The following two data sets came from [this github](https://github.com/unitedstates/congress-legislators/).

In [285]:
current_legislators = pd.read_csv('./loren_data/legislators-current.csv')
current_legislators.shape

(537, 34)

In [286]:
current_leg_cols = list(current_legislators.columns)
current_leg_cols

['last_name',
 'first_name',
 'middle_name',
 'suffix',
 'nickname',
 'full_name',
 'birthday',
 'gender',
 'type',
 'state',
 'district',
 'senate_class',
 'party',
 'url',
 'address',
 'phone',
 'contact_form',
 'rss_url',
 'twitter',
 'facebook',
 'youtube',
 'youtube_id',
 'bioguide_id',
 'thomas_id',
 'opensecrets_id',
 'lis_id',
 'fec_ids',
 'cspan_id',
 'govtrack_id',
 'votesmart_id',
 'ballotpedia_id',
 'washington_post_id',
 'icpsr_id',
 'wikipedia_id']

In [287]:
historical_legislators = pd.read_csv('./loren_data/legislators-historical.csv')
historical_legislators.shape

(12056, 34)

In [288]:
historical_leg_cols = list(historical_legislators.columns)
historical_leg_cols

['last_name',
 'first_name',
 'middle_name',
 'suffix',
 'nickname',
 'full_name',
 'birthday',
 'gender',
 'type',
 'state',
 'district',
 'senate_class',
 'party',
 'url',
 'address',
 'phone',
 'contact_form',
 'rss_url',
 'twitter',
 'facebook',
 'youtube',
 'youtube_id',
 'bioguide_id',
 'thomas_id',
 'opensecrets_id',
 'lis_id',
 'fec_ids',
 'cspan_id',
 'govtrack_id',
 'votesmart_id',
 'ballotpedia_id',
 'washington_post_id',
 'icpsr_id',
 'wikipedia_id']

In [289]:
current_diff_cols = list(set(current_leg_cols).difference(historical_leg_cols))
current_diff_cols

[]

The above determines the columns are the same. We shrank the historical ones to eliminate irrelevant entries. Making birthdays datetime. Referenced [this site](https://www.w3schools.com/python/python_datetime.asp).

In [290]:
historical_legislators['birthday'] = pd.to_datetime(historical_legislators['birthday'], format = '%Y-%m-%d')
current_legislators['birthday'] = pd.to_datetime(current_legislators['birthday'], format = '%Y-%m-%d')
historical_legislators = historical_legislators[historical_legislators['birthday'] > datetime(1922,1, 1)]

relevant_legislators = pd.concat([current_legislators, historical_legislators])
relevant_legislators = relevant_legislators[['last_name', 'first_name', 'middle_name', 'suffix',
                                             'full_name', 'birthday', 'gender', 'type', 'state', 'party', 'district']]

In [291]:
relevant_legislators.dtypes

last_name              object
first_name             object
middle_name            object
suffix                 object
full_name              object
birthday       datetime64[ns]
gender                 object
type                   object
state                  object
party                  object
district              float64
dtype: object

The following is to build a first_name and last_name column in the house dataframe to make it easier to search for the representative in the relevant_legislators dataframe. This worked for the fast majority of representatives/instances, but there were a few representatives for whom it identified the incorrect first and/or last name, as well as some dirty data (i.e. 'None' at the start of the name, which became the first name). That's cleaned below.

In [292]:
house['name'] = house['name'].map(lambda x: x.replace("Hon. ", ""))
house['first_name'] = house['name'].map(lambda x: x.split()[0])
house['last_name'] = house['name'].map(lambda x: x.split()[-1])

There were quite a number of representatives that needed special treatment to get the data from the relevant_representatives dataframe into the house dataframe. 

They were: {'Aston McEachin', 'Christopher Jacobs', 'Cindy Axne', 'Daniel Crenshaw', 'David Cawthorn', 'Debbie Schultz', 'Felix Moore', 'Greg Murphy','Greg Steube','James Banks','James Costa','James Hagedorn', 'James Hill', 'Kenneth Buck', 'Linda Sanchez', 'Michael Gallagher', 'Michael Garcia', 'Mr. Cox', 'Mr. Franklin', 'Mr. Meijer', 'Mrs. Greene', 'Neal FACS', 'Nicholas Taylor', 'None Arenholz', 'None Jacobs', 'None Manning', 'None Newman', 'None Ross', 'None Spartz', 'Richard Allen', 'Rohit Khanna', 'S. Krishnamoorthi', 'Scott Franklin', "Tom O'Halleran"}

We added the second 'if' statement within the for-loop to use last name and district number if first and last name didn't work. For several of the reps this still didn't work. The rewrites to the two dataframes that appear before the for-loop were necessary to address those errors.

Ben Peck was instrumental to solving the error that the for-loop threw when those names were incorrect, and helped with finding and fixing the specific errors at office hours 7/19/22.

In [293]:
house.loc[house['last_name'] == 'FACS', 'last_name'] = 'Dunn'
house.loc[house['last_name'] == 'Arenholz', 'last_name'] = 'Hinson'

relevant_legislators.loc[relevant_legislators['last_name'].str.contains('Halleran'), 'last_name'] = "O'Halleran"
relevant_legislators.loc[relevant_legislators['last_name'] == 'Sánchez', 'last_name'] = 'Sanchez'

In [298]:
house['party'] = ''
house['birthday'] = ''
house['gender'] = ''

name = set()
for i in house.index:
    #try:
    first_name = house.loc[i, 'first_name']
    last_name = house.loc[i, 'last_name']
    party = relevant_legislators[(relevant_legislators['first_name'] == first_name) & (relevant_legislators['last_name'].map(lambda x: x.split()[-1]) == last_name)]['party'].values
    birthday = relevant_legislators[(relevant_legislators['first_name'] == first_name) & (relevant_legislators['last_name'].map(lambda x: x.split()[-1]) == last_name)]['birthday'].values
    gender = relevant_legislators[(relevant_legislators['first_name'] == first_name) & (relevant_legislators['last_name'].map(lambda x: x.split()[-1]) == last_name)]['gender'].values
    if len(party) == 0:
        party = relevant_legislators[(relevant_legislators['district'] == float(house.loc[i, 'represents'][-2:])) & (relevant_legislators['last_name'].map(lambda x: x.split()[-1]) == last_name)]['party'].values
        birthday = relevant_legislators[(relevant_legislators['district'] == float(house.loc[i, 'represents'][-2:])) & (relevant_legislators['last_name'].map(lambda x: x.split()[-1]) == last_name)]['birthday'].values
        gender = relevant_legislators[(relevant_legislators['district'] == float(house.loc[i, 'represents'][-2:])) & (relevant_legislators['last_name'].map(lambda x: x.split()[-1]) == last_name)]['gender'].values
    if len(party) == 0:
        name.add(str(first_name) + ' ' + str(last_name))
    house.loc[i, 'party'] = party[0]
    house.loc[i, 'birthday'] = birthday[0]
    house.loc[i, 'gender'] = gender[0]
    #except:
        #print(i)
        #break
print(len(name))    

0


In [299]:
house['birthday'] = pd.to_datetime(house['birthday'], format = '%Y-%m-%d')
house.drop(columns = ['first_name', 'last_name'], inplace = True)

In [254]:
house.to_csv('./loren_data/clean_house_2022-07-15.csv')